In [47]:
import fiona
import os
import matplotlib.pyplot as plt
from ipywidgets import HBox, Label, IntSlider,IntRangeSlider
from ipywidgets import HTML
from IPython.display import clear_output
from ipyleaflet import Map, basemaps, basemap_to_tiles, CircleMarker, Popup, Polyline,LayerGroup, LayersControl,ScaleControl, FullScreenControl
from ipyleaflet import WidgetControl
from ipywidgets import interactive
import matplotlib as mpl
import ipywidgets as widgets
from ipywidgets import HBox, Label, IntSlider,IntRangeSlider
import matplotlib.patches as patches
from matplotlib.patches import Polygon
import matplotlib
import matplotlib.colors
import contextily as cx
import shapely.geometry as sg
from rijksdriehoek import rijksdriehoek
from matplotlib.collections import PatchCollection

RD = rijksdriehoek.Rijksdriehoek()
plt.rcParams.update({'font.size': 24})

In [23]:
def read_gpkg(filename):
    
    assert '.' in filename, f'Filename {filename} does not have an extension!'    
    if filename.split('.')[1] != 'gpkg':
        raise ValueError(f'Expected gpkg, got {filename}')
    if not os.path.isfile(f'Data/{filename}'):
        raise ValueError(f'File {filename} not found in Data directory!')
    
    AOI = {}
    with fiona.open(f"Data/{filename}") as layer:
        for feature in layer:     
            AOI[feature['properties']['ogc_fid']] = {
                "crd_dec": [feature['geometry']['coordinates'][ii] for ii in range(len(feature['geometry']['coordinates']))]
                }
            for key in ['h0', 's_h0', 'xI', 's_xI', 'xP', 's_xP', 'xE', 's_xE', 'tau', 's_tau',
                        'rho_xIh0', 'rho_xPh0', 'rho_xEh0', 'rho_tauh0',
                        'rho_xPxI', 'rho_xExI', 'rho_xItau', 
                        'rho_xPxE', 'rho_xPtau', 
                        'rho_xEtau', 'xI_frac']:
                AOI[feature['properties']['ogc_fid']][key] = feature['properties'][key]
    return AOI

def convert_wgs84_to_rd(list_of_coords):
    """
    Converts a list of coordinates in WGS84 to RD
    :param list_of_coords: list of (lon, lat) objects
    :return: list of (rd_x, rd_y) objects
    """
    rd_coords = []
    for coord in list_of_coords:
        RD.from_wgs(lon=coord[0], lat=coord[1])
        rd_coords.append([RD.rd_x, RD.rd_y])
    return rd_coords

In [11]:
gpkg_data = read_gpkg('InSAR_ALS_GNSS_Grav_Leveling.gpkg')

In [67]:
#@widgets.interact_manual(plot=['xI', 's_xI', 'h0', 's_h0'])
def plot_overview(plot='xI'):
    with output:
        clear_output(wait=True)
        display(main_plot)
    if plot != 'Select':
        with output:
            filename = "Data/AoI_GreenHeart-polygon.shp"
            shape = fiona.open(filename)
            finished = False
            shape_iter = iter(shape)
            aoi_rd_poly = None
            while not finished:
                try:
                    shp = next(shape_iter)
                    polygons = [sg.Polygon(convert_wgs84_to_rd(shp["geometry"]["coordinates"][ii])) for ii in range(len(shp["geometry"]["coordinates"]))]
                    if len(polygons) > 1 or aoi_rd_poly is not None:
                        raise ValueError("Cannot handle AOI with more than 1 polygon!")
                    aoi_rd_poly = polygons[0]
                    if not aoi_rd_poly.is_valid:
                        raise ValueError("Invalid polygon AOI!")
                except StopIteration:
                    finished = True
            if aoi_rd_poly is None:
                raise ValueError(f"No valid AOI polygon found in {filename}!")
    
            eups = list(sorted(list(gpkg_data.keys())))
    
            fig, ax = plt.subplots(figsize=(12,10))
            ax.plot(aoi_rd_poly.exterior.coords.xy[0], aoi_rd_poly.exterior.coords.xy[1], c='k', linestyle='--')
            ax.grid(True)
            patches = []
            patches_edge = []
            vals = []
            mean_crd = []
            crds = []
    
    
            for EUP in gpkg_data.keys():
                    #if final_InSAR_EUPs[eups[EUP]]['xI'] != -999:
                    patches.append(Polygon(gpkg_data[EUP]['crd_dec'][0]))
                    patches_edge.append(Polygon(gpkg_data[EUP]['crd_dec'][0], lw=0.5, edgecolor='black'))
                    if plot in ['xI', 's_xI']:
                        vals.append(gpkg_data[EUP][plot] * gpkg_data[EUP]['xI_frac'] * 365.2425 * 1000)
                    elif plot in ['h0']:
                        vals.append(gpkg_data[EUP][plot])
                    elif plot in ['s_h0']:
                        vals.append(gpkg_data[EUP][plot] * 100)
                    else:
                        vals.append(gpkg_data[EUP][plot] * 1000)
            
            p = PatchCollection(patches, cmap = matplotlib.cm.bwr_r)
            p.set_array(vals)
            
            
    
            cbar = ax.colorbar(p)
            if plot == 'xI':
                cbar.set_label('Mean Yearly Irreversible Subsidence [mm/y]')
                p.set_clim(-10, 10)
            elif plot == 'h0':
                cbar.set_label('Elevation on 1 January 2015 [m]')
                p.set_clim(-8, 2)
    
            if plot == 's_xI':
                cbar.set_label('Mean Yearly Irreversible Subsidence Standard Deviation [mm/y]')
                p.set_clim(0, 10)
            elif plot == 's_h0':
                cbar.set_label('Elevation on 1 January 2015 Standard Deviation [cm]')
                p.set_clim(0, 10)
    
            ax.add_collection(p)
            ax.set_aspect(aspect=1)
            
            ax.set_xticks([])
            ax.set_yticks([])
            cx.add_basemap(ax, crs='epsg:28992', source=cx.providers.OpenStreetMap.Mapnik)
            plt.show()

In [65]:
plot_drop = widgets.Dropdown(
        options=['Select', 'xI', 's_xI', 'h0', 's_h0'],
        value='Select',
        description='Parameter:',
        disabled=False,
    )
main_plot = interactive(plot_overview, plot=plot_drop)

In [66]:
button = widgets.Button(description="Run Me!")
output = widgets.Output()

display(button, output)

def on_button_clicked(b):
    with output:
        clear_output()
        display(main_plot)
        #plot_overview(plot_drop.value)

#     m
#     print(type(m)

button.on_click(on_button_clicked)

Button(description='Run Me!', style=ButtonStyle())

Output()